## Pre processing

In [1]:
import pandas as pd
import pyomo.environ as pyo
import numpy as np

In [2]:
# Read CSV file, disaggregated demand (3 consumers)
cons_df = pd.read_csv("Summer_Berlin_Disaggregated_3Consumers.csv", sep=';')

In [3]:
# Identify the 3 consumer columns and check if they exist in the excel file.
consumer_cols = ["consumer_1", "consumer_2", "consumer_3"]
for c in consumer_cols:
    assert c in cons_df.columns, f"Missing column {c} in disaggregated file."

# Define T as the number of time steps based on the length of the demand dataframe
T = len(cons_df)
t_idx = range(T)

# The D_ct dictionary is currently not used in the Pyomo model for m.D,
# as m.D uses an aggregate demand. It can be ignored or removed for now.
# D_ct = {(c, t): float(cons_df.loc[t, c]) for c in consumer_cols for t in t_idx}

In [4]:
# Read CSV file : PV and market price and check the number of rows
pv_df = pd.read_csv("Berlin_Summer_Week_01.07_07.07_2023.csv")
price_df = pd.read_csv("Hourly_Market_Prize_Berlin_Summer_Week.csv")

assert len(pv_df) == T, "PV file length != demand length"
assert len(price_df) == T, "Price file length != demand length"

In [5]:
# Build (c,t) demand dictionary for Pyomo Param ( instead of a vector we build a matric , note : this is only for demand
# because [we have consumer and time])
D_ct = {(c, t): float(cons_df.loc[t, c]) for c in consumer_cols for t in t_idx}

In [6]:
# PV scaling : W -> kW for reference plant, # kW per 1 kWp installed
pv_ref_kwp = 1000.0
pv_df["pv_kw"] = pv_df["Power"].astype(float) / 1000.0
pv_df["pv_per_kwp"] = pv_df["pv_kw"] / pv_ref_kwp
# convert to vector for PV and market price
PV_unit = pv_df["pv_per_kwp"].to_numpy(dtype=float)
P_buy   = price_df["price_eur_per_kwh"].to_numpy(dtype=float)
# build new vector
sell_factor = 0.50
P_sell = sell_factor * P_buy


In [8]:
# fix the size of PV and battery
PV_cap_opt = 2429.59124087591
E_cap_opt  = 177.769887208029

# Battery technical parameters
eta = 0.90
soc_min_frac  = 0.10
soc_max_frac  = 0.90
soc_init_frac = 0.50
soc_final_frac= 0.50
c_rate = 1.0
P_cap_opt = c_rate * E_cap_opt  # 1C
# R1 fixed equal shares
alpha = {c: 1.0/len(consumer_cols) for c in consumer_cols}

## Building the model

In [9]:
m = pyo.ConcreteModel()

In [10]:
# Sets
m.T = pyo.RangeSet(0, T-1)
m.Tsoc = pyo.RangeSet(0, T)
# defining a set for consumers
m.C = pyo.Set(initialize=consumer_cols)

In [11]:
#parameters :
m.D = pyo.Param(m.C, m.T, initialize=D_ct)
m.PV_unit = pyo.Param(m.T, initialize={t: float(PV_unit[t]) for t in t_idx})
m.p_buy   = pyo.Param(m.T, initialize={t: float(P_buy[t]) for t in t_idx})
m.p_sell  = pyo.Param(m.T, initialize={t: float(P_sell[t]) for t in t_idx})

# Fixed sizes as Params
m.PV_cap = pyo.Param(initialize=float(PV_cap_opt))
m.E_cap  = pyo.Param(initialize=float(E_cap_opt))
m.P_cap  = pyo.Param(initialize=float(P_cap_opt))

# The constant of equal shares .
m.alpha  = pyo.Param(m.C, initialize={c: float(alpha[c]) for c in consumer_cols})


In [12]:
#Decision Variables
# PV allocations (consumer-level for R1)
m.pv2load = pyo.Var(m.C, m.T, domain=pyo.NonNegativeReals)
m.pv2batt = pyo.Var(m.C, m.T, domain=pyo.NonNegativeReals)
m.pv_exp  = pyo.Var(m.T, domain=pyo.NonNegativeReals)

# Grid to load and grid to (virtual) battery per consumer
m.g2load  = pyo.Var(m.C, m.T, domain=pyo.NonNegativeReals)
m.g2batt  = pyo.Var(m.C, m.T, domain=pyo.NonNegativeReals)

# Virtual battery operation per consumer
m.ch  = pyo.Var(m.C, m.T, domain=pyo.NonNegativeReals)
m.dis = pyo.Var(m.C, m.T, domain=pyo.NonNegativeReals)
m.soc = pyo.Var(m.C, m.Tsoc, domain=pyo.NonNegativeReals)

# Binary (no simultaneous charge/discharge)
m.u = pyo.Var(m.T, domain=pyo.Binary)

In [13]:
#Constraint
# PV generation each hour (expression)
m.pv_gen = pyo.Expression(m.T, rule=lambda m, t: m.PV_unit[t] * m.PV_cap)


# (A) PV split (total)
def pv_split_rule(m, t):
    return (sum(m.pv2load[c,t] + m.pv2batt[c,t] for c in m.C) + m.pv_exp[t]
            == m.pv_gen[t])
m.pv_split = pyo.Constraint(m.T, rule=pv_split_rule)

# (B) R1 PV allocation limit: each consumer can't exceed its equal share of PV (load+charge)
def pv_share_limit_rule(m, c, t):
    return m.pv2load[c,t] + m.pv2batt[c,t] <= m.alpha[c] * m.pv_gen[t]
m.pv_share_limit = pyo.Constraint(m.C, m.T, rule=pv_share_limit_rule)

# (C) Demand balance per consumer
def demand_balance_rule(m, c, t):
    return m.pv2load[c,t] + m.dis[c,t] + m.g2load[c,t] == m.D[c,t]
m.demand_balance = pyo.Constraint(m.C, m.T, rule=demand_balance_rule)

# (D) Charging definition per consumer (virtual battery)
def charge_def_rule(m, c, t):
    return m.ch[c,t] == m.pv2batt[c,t] + m.g2batt[c,t]
m.charge_def = pyo.Constraint(m.C, m.T, rule=charge_def_rule)

# (E) SOC dynamics per consumer
def soc_update_rule(m, c, t):
    return m.soc[c,t+1] == m.soc[c,t] + eta*m.ch[c,t] - (1/eta)*m.dis[c,t]
m.soc_update = pyo.Constraint(m.C, m.T, rule=soc_update_rule)

# (F) SOC bounds per consumer using virtual capacity alpha*E_cap
def soc_min_rule(m, c, k):
    return m.soc[c,k] >= soc_min_frac * (m.alpha[c] * m.E_cap)
def soc_max_rule(m, c, k):
    return m.soc[c,k] <= soc_max_frac * (m.alpha[c] * m.E_cap)
m.soc_min = pyo.Constraint(m.C, m.Tsoc, rule=soc_min_rule)
m.soc_max = pyo.Constraint(m.C, m.Tsoc, rule=soc_max_rule)

# (G) Initial/final SOC per consumer (virtual capacity)
m.soc_init  = pyo.Constraint(m.C, rule=lambda m,c: m.soc[c,0] == soc_init_frac  * (m.alpha[c]*m.E_cap))
m.soc_final = pyo.Constraint(m.C, rule=lambda m,c: m.soc[c,T] == soc_final_frac * (m.alpha[c]*m.E_cap))

# (H) Virtual battery power limits with coherency (one binary per hour)
# Each consumer has virtual power alpha*P_cap, and all must be in same mode via u[t]
def ch_onoff_rule(m, c, t):
    return m.ch[c,t] <= m.u[t] * (m.alpha[c] * m.P_cap)
def dis_onoff_rule(m, c, t):
    return m.dis[c,t] <= (1 - m.u[t]) * (m.alpha[c] * m.P_cap)
m.ch_onoff  = pyo.Constraint(m.C, m.T, rule=ch_onoff_rule)
m.dis_onoff = pyo.Constraint(m.C, m.T, rule=dis_onoff_rule)





In [15]:
#Objective function
# fee imposed for grid network services
grid_fee_eur_per_kwh = 0.0

def op_cost_rule(m):
    grid_cost = sum((m.p_buy[t] + grid_fee_eur_per_kwh) *
                    (sum(m.g2load[c,t] + m.g2batt[c,t] for c in m.C))
                    for t in m.T)
    export_revenue = sum(m.p_sell[t] * m.pv_exp[t] for t in m.T)
    return grid_cost - export_revenue

m.obj = pyo.Objective(rule=op_cost_rule, sense=pyo.minimize)

This is usually indicative of a modelling error.
To avoid this warning, use block.del_component() and block.add_component().


In [16]:
import pyomo.environ as pyo

# Install glpk-utils if not already installed
!apt-get -qq update
!apt-get -qq install -y glpk-utils

# Configure the GLPK solver with the explicit path to its executable
solver = pyo.SolverFactory("glpk", executable="/usr/bin/glpsol")

# Solve the model
res = solver.solve(m)

# Print the operational cost
print("Operational cost (€) =", pyo.value(m.obj))

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Selecting previously unselected package libsuitesparseconfig5:amd64.
(Reading database ... 117528 files and directories currently installed.)
Preparing to unpack .../libsuitesparseconfig5_1%3a5.10.1+dfsg-4build1_amd64.deb ...
Unpacking libsuitesparseconfig5:amd64 (1:5.10.1+dfsg-4build1) ...
Selecting previously unselected package libamd2:amd64.
Preparing to unpack .../libamd2_1%3a5.10.1+dfsg-4build1_amd64.deb ...
Unpacking libamd2:amd64 (1:5.10.1+dfsg-4build1) ...
Selecting previously unselected package libcolamd2:amd64.
Preparing to unpack .../libcolamd2_1%3a5.10.1+dfsg-4build1_amd64.deb ...
Unpacking libcolamd2:amd64 (1:5.10.1+dfsg-4build1) ...
Selecting previously unselected package libglpk40:amd64.
Preparing to unpack .../libglpk40_5.0-1_amd64.deb ...
Unpacking libglpk40:amd64 (5.0-1) ...
Selecti